# Preparation for creating simulated data
## Aim of this notebook
This notebook contains the preparation required to run amplicon-sequencing (AmpSeq) read simulations using the read simulation pipeline "pop_var_sim", provided in a [separate repository](https://github.com/ISOinaBox/pop_var_sim).  

While the files produced in this notebook can be used to create simulated reads, the main use of this notebook is to serve as a template and guide for generating simulation runs, tailor-made for a pipeline that is to be validated.  

This notebook focuses on AmpSeq data because simulated data is particularly useful for this type of data. Whole-genome data for pipeline validataion can be obtained from real-world datasets that are readily available [from public resources](https://www.malariagen.net/resource/36/). AmpSeq data, on the other hand, depends on a specific primer panel, for which data may not be readily available. In addition, a pipeline written and validated for a specific AmpSeq panel, such as [SpotMalaria](https://ngs.sanger.ac.uk/production/malaria/Resource/29/20200705-GenRe-04a-SpotMalaria-0.39.pdf) may need to be validated against other primer panels, for which data does not exist yet.  

In part 1 of this notebook, we obtain read counts from high-quality public data. This step is necessary for any simulation, so that the generated data provides realistic input for an analysis pipeline. Here, we simulate a [SpotMalaria](https://ngs.sanger.ac.uk/production/malaria/Resource/29/20200705-GenRe-04a-SpotMalaria-0.39.pdf) AmpSeq run, for which real-world samples exist in public repositories. This may not always be the case for simulating different AmpSeq primer panel runs, in which case the user may need to estimate realistic read counts in different ways.

In part 2, the simulation run input files are created. This demonstrates how genome data and publicly available data on relevant mutations can be used to simulate WT and mutant strain results and should be modified according to the validation that is to be carried out. For example, a new pipeline might need to be tested in terms of how robustly it can call a mutant allele in a mix of WT and mutant parasites, which can be configured in the simulation pipeline run.  

## Part 1: read counts to guide simulation
An important consideration for a simulated dataset is the number of reads to create. This should be within the range of expected read numbers from real-world experiments because unrealistically high or low read counts will not provide useful insights into the validity of pipeline results.  

In this example, we are simulating reads for a wild-type and a mutant _P. falciparum_ strain. We will simulate an AmpSeq run using the [SpotMalaria](https://www.malariagen.net/project/spotmalaria/) panel, as used for the [GenRe Mekong project](https://www.malariagen.net/resource/29/) project. 

This primer panel consists of three sub-panels: SPEC (speciation), GRC1 and GRC2.  
The data for these panels is provided in [this spreadsheet](https://www.malariagen.net/wp-content/uploads/2023/11/20200705-GenRe-04b-SpotMalaria-SupplementaryFile1.xls).  

As noted in the introduction, this analysis is specific to the SpotMalaria AmpSeq primer panel and will need to be modified - based on the template provided here - to accommodate other AmpSeq panels or whole-genome data.  

### FASTQ data
The FASTQ files used in this notebook are based on the ENA data download manifests created in [this notebook](../real-world_gold-standard_data/Pf8-GenReMekong/pf8genre.ipynb). But __note__ that the FASTQ files are __not__ icluded in this repo because of the large amount of data contained in them. If you would like to re-run this notebook, please refer to the explanation provided [here](../real-world_gold-standard_data/Pf8-GenReMekong/pf8genre.ipynb#Example:-how-to-use-the-downloader-tool) to download the FASTQ files from ENA using the downloader tool provided in this repo.

For the purpose of this analysis, the dataset [Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv](../real-world_gold-standard_data/Pf8-GenReMekong/Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv), created [here](../real-world_gold-standard_data/Pf8-GenReMekong/pf8genre.ipynb#Save-results:-concordant-phenotype-data) was used. This contains a subset of 907 samples of all high-quality data from the GenRe Mekong project where drug resistance phenotype agrees with the WGS sample analysis in Pf8.  

### Creating the read counts data file
A file "fastq_readcounts.csv" is already prepared in this directory and used by this notebook. 

It is recommended to use that file as provided in this folder. The following instructions are only needed to re-create that file and to document how it was generated. 

Ensure that the following Python packages are installed on your system:
- requests
- pandas

Then, in a terminal, run the following commands. The commands can be found alongside this notebook.   
Both commands will take a potentially long time to run (depending on your system and network) and the first command will download a large amount of data over the internet. If you are working in a HPC/cloud environment, you may want to split the data into chunks and parallelise these tasks. This is beyond the scope of this recipe.  

__Step 1: Download FASTQ files__    
Run the following command in a terminal (relative paths from this folder):
```bash
python ../../../real-world_gold-standard_data/Pf8-GenReMekong/scripts/ENA_data_helper.py  \
    download \
    --data ../../../real-world_gold-standard_data/Pf8-GenReMekong/additional_output_files/ENA_download_manifest_concordant_phenotype_high_quality_samples.csv  \
    --skip_errors \
    --download_attempts 10 \
    --out ENA_download_concordant_phenotype_high_quality_sample \
```
It will create a folder "ENA_download_concordant_phenotype_high_quality_sample" in this directory. The folder contains the 3628 FASTQ (gzipped) files belonging to the [Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv](../real-world_gold-standard_data/Pf8-GenReMekong/Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv), created [here](../../../real-world_gold-standard_data/Pf8-GenReMekong/scripts/pf8genre.ipynb#Save-results:-concordant-phenotype-data).  

__Step 2: Obtain read counts__  
Once the FASTQ download has completed, use the script "run_counts.sh", also in this folder, like this:  
(or change the path to the downloaded FASTQ files if different)

```bash
./run_counts.sh ENA_download_concordant_phenotype_high_quality_sample
```  
This will create a file "fastq_readcounts.csv" with file names and read counts for all mate1 files.

### Calculate AmpSeq panel stats
Using the dataset [Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv](../real-world_gold-standard_data/Pf8-GenReMekong/Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv) and the pre-computed (see above) csv file of read counts, create read count stats for the three SpotMalaria amplicon panels.  

In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [2]:
counts_df = pd.read_csv('fastq_readcounts.csv')
counts_df

,file,count
0,ERR14388603_1.fastq.gz,32819
1,ERR14388604_1.fastq.gz,29409
2,ERR14388605_1.fastq.gz,1094
3,ERR14388606_1.fastq.gz,38189
4,ERR14388607_1.fastq.gz,35804
...,...,...
3623,ERR15628752_1.fastq.gz,18788821
3624,ERR15628753_1.fastq.gz,13952241
3625,ERR15628764_1.fastq.gz,14159323
3626,ERR15628798_1.fastq.gz,7910579


In [3]:
samples_df = pd.read_csv('../real-world_gold-standard_data/Pf8-GenReMekong/Pf8-GenReMekong_concordant_phenotype_high_quality_samples.csv')
samples_df

,sample,Artemisinin,Piperaquine,Mefloquine,Chloroquine,Pyrimethamine,Sulfadoxine,DHA-PPQ,AS-MQ,Species,ENA,% callable,ENA_acc_Pf8,ENA_acc_GenRe_GRC1,ENA_acc_GenRe_GRC2,ENA_acc_GenRe_SPEC
0,RCN12025,Resistant,Sensitive,Sensitive,Resistant,Resistant,Resistant,Sensitive,Sensitive,Pf,ERR3409305,89.14,ERR15625306,ERR14388603,ERR14388604,ERR14388605
1,RCN12026,Resistant,Resistant,Sensitive,Resistant,Resistant,Resistant,Resistant,Sensitive,Pf,ERR3409312,85.83,ERR15625307,ERR14388606,ERR14388607,ERR14388608
2,RCN12028,Resistant,Resistant,Sensitive,Resistant,Resistant,Resistant,Resistant,Sensitive,Pf,ERR3409373,87.23,ERR15625309,ERR14388612,ERR14388613,ERR14388614
3,RCN12031,Resistant,Resistant,Sensitive,Resistant,Resistant,Resistant,Resistant,Sensitive,Pf,ERR3409314,87.04,ERR15625310,ERR14388621,ERR14388622,ERR14388623
4,RCN12032,Resistant,Sensitive,Sensitive,Resistant,Resistant,Resistant,Sensitive,Sensitive,Pf,ERR3409311,88.65,ERR15625311,ERR14388624,ERR14388625,ERR14388626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,RCN26767,Resistant,Sensitive,Sensitive,Resistant,Resistant,Resistant,Sensitive,Sensitive,Pf,ERR7221085,87.92,ERR15628752,ERR14397546,ERR14397547,ERR14397548
903,RCN26775,Sensitive,Sensitive,Sensitive,Resistant,Resistant,Sensitive,Sensitive,Sensitive,Pf,ERR7221086,87.30,ERR15628753,ERR14397570,ERR14397571,ERR14397572
904,RCN26820,Resistant,Sensitive,Sensitive,Resistant,Resistant,Resistant,Sensitive,Sensitive,Pf,ERR7221102,89.35,ERR15628764,ERR14397663,ERR14397664,ERR14397665
905,RCN26932,Resistant,Sensitive,Sensitive,Resistant,Resistant,Resistant,Sensitive,Sensitive,Pf,ERR7221141,86.03,ERR15628798,ERR14397975,ERR14397976,ERR14397977


Read counts in ```counts_df``` are assigned to file names that do not contain the name of the primer panel to which the file belongs. Use the ```samples_df``` data to assign the panel name (SPEC, GRC1 or GRC2) to each read count by looking up the file name.  
Drop Pf8 (WGS) data, which is not relevant here.

In [4]:
counts_df['panel']=counts_df.apply(
    lambda row: 
    'SPEC' if samples_df.GenRe_SPEC_ENA_FASTQ_FTP_1.str.contains(row.file).any() else 
    'GRC1' if samples_df.GenRe_GRC1_ENA_FASTQ_FTP_1.str.contains(row.file).any() else 
    'GRC2' if samples_df.GenRe_GRC2_ENA_FASTQ_FTP_1.str.contains(row.file).any() else 
    'Pf8' if samples_df.Pf8_ENA_FASTQ_FTP_1.str.contains(row.file).any() else None, axis=1)
counts_df = counts_df[ counts_df['panel'].isin(['SPEC','GRC1','GRC2'])]

AttributeError: 'DataFrame' object has no attribute 'GenRe_SPEC_ENA_FASTQ_FTP_1'

Sanity check: we should have 907 samples for each GenRe primer panel

In [ ]:
counts_df.groupby('panel').size().to_frame('count').reset_index()

Calculate stats for read counts per primer panel: minimum, maximum, mean and percentiles at 25%, 50% and 75%.  
These will be used to guide the number of reads to simulate for each panel

In [ ]:
def q25(x):
    return x.quantile(0.25)
def q50(x):
    return x.quantile(0.5)
def q75(x):
    return x.quantile(0.75)

counts_df.groupby('panel').agg({'count': ['mean', 'min','max', q25,q50,q75]})

### Conclusion
The read count stats shown above can be used as guidelines for simulation runs. Specifically, the 25, 50 and 75 quantile are useful guides for three datasets to be created.  

## Part 2: Configuring the read simulation pipeline
The [read simulation pipeline](https://github.com/ISOinaBox/pop_var_sim) requires a set of input files to configure the simulation run. This part is about creating these files.

### Create a bed-like file of primer positions
A file of primer positions in a bed-like format is rquired to configure start/end positions of amplicons to simulate.  
As we are simulating a SpotMalaria AmpSeq run, we are using primer positions for the GenRe-04b-SpotMalaria AmpSeq panel, available from www.malariagen.net in form of an Excel file. 

In [ ]:
panel_df = pd.read_excel(
    "https://www.malariagen.net/wp-content/uploads/2023/11/20200705-GenRe-04b-SpotMalaria-SupplementaryFile1.xlsx",
    sheet_name="P. falciparum amplicon primers"
)

Change column names in line with expected headers for bed-like config file for the simulation pipeline.  
Add a column DESC, expected by the simulation pipeline. Use it to record the primer panel ('GRC1','GRC2' or 'SPEC') and drop rows (if any) that do not have a primer panel assigned.  
Only keep the columns relevant to the simulation pipeline.

In [ ]:
panel_df.rename(
    columns={
        'Chromosome':'#CHROM', 
        'Amplicon Start Position':'START',
        'Amplicon Stop Position':'END'
    },inplace=True)

panel_df['DESC'] = panel_df.apply(
    lambda row:
    'SPEC' if 'SPEC' in row['Multiplex'] else 
    'GRC1' if 'GRC1' in row['Multiplex'] else 
    'GRC2' if 'GRC2' in row['Multiplex'] else
    np.NaN, 
    axis=1
)
panel_df=panel_df[panel_df['DESC'].notna()]
keep_cols=['#CHROM','START','END','DESC']
panel_df=panel_df[keep_cols]

Coordinates are 1-based in this file but need to be converted to 0-based for the simulation pipeline.

In [ ]:
panel_df['START']=pd.to_numeric(panel_df['START'],errors='coerce')-1

In [ ]:
panel_df['END']=pd.to_numeric(panel_df['END'],errors='coerce')-1

Keep only rows where primer start and end is provided

In [ ]:
panel_df = panel_df[(panel_df['START'].notna()) & (panel_df['END'].notna())]

convert positions to integer

In [ ]:
panel_df['START']=panel_df['START'].astype(np.int64)

In [ ]:
panel_df['END']=panel_df['END'].astype(np.int64)

In [ ]:
panel_df

Create versions of the panel file for each of the three sub-panels and save files.

In [ ]:
spec_panel_df = panel_df[panel_df['DESC']=='SPEC']
grc1_panel_df = panel_df[panel_df['DESC']=='GRC1']
grc2_panel_df = panel_df[panel_df['DESC']=='GRC2']

In [ ]:
spec_panel_df.to_csv('SpotMalaria-SPEC_Pf_amplicon_primers.bed', index=False, sep="\t")
grc1_panel_df.to_csv('SpotMalaria-GRC1_Pf_amplicon_primers.bed', index=False, sep="\t")
grc2_panel_df.to_csv('SpotMalaria-GRC2_Pf_amplicon_primers.bed', index=False, sep="\t")

### Reference genome and annotation
The read simulation pipeline requires a reference genome. The _P. falciparum_ 3D7 reference genome was obtained from 
https://plasmodb.org/common/downloads/Current_Release/Pfalciparum3D7/fasta/data/PlasmoDB-68_Pfalciparum3D7_Genome.fasta
and is provided in this folder.  
In addition, a GFF genome annotation file is required for the next step, creating a variant file. It was obtained from 
https://plasmodb.org/a/service/raw-files/release-68/Pfalciparum3D7/gff/data/PlasmoDB-68_Pfalciparum3D7.gff and is also provided in this folder.


### Variant file
This section creates an example variants file for the read simulation pipeline. It can be used to simulate a run of the SpotMalaria panel with a genome that carries a resistance genotype. The simulation pipeline requires a vcf-like file with SNP data in order to create a mutant genome from the reference WT.

As pointed out in the introduction, this is intended as a template for creating tailor-made simulated read datasets. In this example, reads will be created to simulate a chloroquine resistance encoded in the crt gene (PF3D7_0709000). This could be used to verify that a pipeline correctly calls this mutation and the resistance phenotype from the simulated read data.  

Drug-resistance genotypes are recorded in the Pf8_resistance_classification.pdf file available [here](https://pf8-release.cog.sanger.ac.uk/Pf8_resistance_classification.pdf).  

According to page 3 of this document, the mutation PF3D7_0709000:K76T leads to chloroquine resistance.  

To create the variants (vcf-like) file for the simualtion pipeline, we need to convert this aminoacid position into a genome position and apply a SNP at DNA level.  

A python module is provided in this folder, which can be used stand-alone but will be used via imported functions here in order to convert from gene/AA to genomic NA coordinates via a GFF annotation file.

In [ ]:
from snp2mutant_coords import find_cds_for_gene, aa_to_genomic_from_exons, sequence_for_pos

In [ ]:
gene_id='PF3D7_0709000'
aa_pos=76
gff_file='PlasmoDB-68_Pfalciparum3D7.gff'

chrom, strand, exons, diagnostics = find_cds_for_gene(gff_file, gene_id)
start, end = aa_to_genomic_from_exons(exons, aa_pos, strand)

print(f'{gene_id} AA pos {str(aa_pos)} = genomic position (1-based) {chrom}:{str(start)}-{str(end)}')

In [ ]:
seq=sequence_for_pos(
    'PlasmoDB-68_Pfalciparum3D7_Genome.fasta',
    chrom, 
    strand,
    start,
    end)
print(f'sequence at {chrom}:{str(start)}-{str(end)}:{seq}')

__RESULT:__ 
To create the chloroquine resistance mutaion PF3D7_0709000:K76T, codon 'AAA' at genomic position f3D7_07_v3:403624-403626 (encoding Lys/K) needs to be changed to 'ACA' to encode Thr/T. For reference see https://en.wikipedia.org/wiki/DNA_and_RNA_codon_tables.  

The vcf-like file for this mutation looks like this:
```
#CHROM	POS	REF	ALT
Pf3D7_07_v3	403625	A	C
```
This has been saved in this folder as file ```Pf3D7_07_v3_403625_a2c```

### Simulation pipeline final input files
With the file created so far in this section, we are now able to create the final input files for a run of the [read simulation pipeline](https://github.com/ISOinaBox/pop_var_sim).  
#### Haplotype manifest
According to the [pipeline manual](https://github.com/ISOinaBox/iib_readsim/blob/main/README.md), a "haplotype manifest" csv file is required to configure the reference genome(s) on which to create simulated reads. As we have three primer panels and two genotypes (WT and crt mutant), we need one line per genotype and primer panel in the manifest as shown here and saved as file ```hap_manifest_example.csv``` in this folder:  
```
haplotype,base_fasta,variants_file,bed_file
wt_spec,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,null,SpotMalaria-SPEC_Pf_amplicon_primers.bed
wt_grc1,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,null,SpotMalaria-GRC1_Pf_amplicon_primers.bed
wt_grc2,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,null,SpotMalaria-GRC2_Pf_amplicon_primers.bed
crtmut_spec,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,Pf3D7_07_v3_403625_a2c,SpotMalaria-SPEC_Pf_amplicon_primers.bed
crtmut_grc1,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,Pf3D7_07_v3_403625_a2c,SpotMalaria-GRC1_Pf_amplicon_primers.bed
crtmut_grc2,PlasmoDB-68_Pfalciparum3D7_Genome.fasta,Pf3D7_07_v3_403625_a2c,SpotMalaria-GRC2_Pf_amplicon_primers.bed
```

__Explanation of the file__:  
Line 1 configures a WT genotype, using the 3D7 genome provided in this folder. The value in the 'variants_file' column is 'none' because we use the 3D7 genome as-is. The 'bed_file' is the file created earlier, providing positions of the primers in the SpotMalaria SPEC sub-panel.  Lines 2 and 3 do the same but refer to the GRC1 and GRC2 bed files, respectively.  
Lines 4-6 configure the mutant genotype for the crt mutation as discussed in detail in the previous section. Here, we use the same 3D7 reference genome, but provide the vcf-like file for a single SNP that re-creates the resistant mutant, which was produced in the previous section. The bed (primer position) files are the same as for lines 1-3. 

#### Sample manifest
The sample manifest file defines the samples that are to be simulated. Each sample can contain one or more haplotypes, as defined in the above haplotype manifest file. This would enable the simulation of mixed WT/mutant samples, which could be used to test the robustness of a pipeline for heterozygous results.  
In the example provided here, we are not creating such mixed samples. Instead, we configure the pipeline to create one sample for each genotype (WT/mut) and for each primer sub-panel.  
The read numbers are taken from the earlier analysis of real-world read counts in this notebook. For each genotype/primer-panel combination, we simulate one run each with the 25, 50 and 75 quantile of read counts from part 1 of this notebook.  
The final file, which is available in this folder as ```sample_manifest_example.json``` has the following content:  
```
[
    {
        "sample_id": "crtmut_spec_q25", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 654
    },
    {
        "sample_id": "wt_spec_q50", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 988
    },
    {
        "sample_id": "wt_spec_q75", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 1445
    },
    {
        "sample_id": "wt_grc1_q25", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 25568
    },
    {
        "sample_id": "wt_grc1_q50", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 32424
    },
    {
        "sample_id": "wt_grc1_q75", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 40281
    },
        {
        "sample_id": "wt_grc2_q25", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 26732
    },
    {
        "sample_id": "wt_grc2_q50", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 32519
    },
    {
        "sample_id": "wt_grc2_q75", 
        "genotypes": ["wt_spec"], 
        "proportions": [1], 
        "num_reads": 39864
    },

    {
        "sample_id": "crtmut_spec_q25", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 654
    },
    {
        "sample_id": "crtmut_spec_q50", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 988
    },
    {
        "sample_id": "crtmut_spec_q75", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 1445
    },
    {
        "sample_id": "crtmut_grc1_q25", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 25568
    },
    {
        "sample_id": "crtmut_grc1_q50", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 32424
    },
    {
        "sample_id": "crtmut_grc1_q75", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 40281
    },
        {
        "sample_id": "crtmut_grc2_q25", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 26732
    },
    {
        "sample_id": "crtmut_grc2_q50", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 32519
    },
    {
        "sample_id": "crtmut_grc2_q75", 
        "genotypes": ["crtmut_spec"], 
        "proportions": [1], 
        "num_reads": 39864
    }

]

```
As an example, sample "crtmut_grc2_q75" uses the crt mutant genotype to simulate a run of the GRC2 panel with 39864 reads, which is the 75 quantile read count for the GRC2 panel as calculated from real-world data in part 1 of this notebook.